# Load Libaries

In [1]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import time
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
import glob
import pickle
import warnings
from scipy import misc
import scipy.ndimage
from skimage import feature
from PIL import Image
import os
import cv2
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Dropout, Flatten, MaxPooling2D
from keras.models import model_from_json
from keras.callbacks import EarlyStopping
from keras import backend as K

Using TensorFlow backend.


# Load training and test data

In [7]:
def get_train_and_test_data(procent, model_number, augmented_data_list):
    train_hollow_folder = "potato_dataset/data_" + str(procent) + "/model_" + str(model_number) + "/train/hollow/"
    train_metal_folder = "potato_dataset/data_" + str(procent) + "/model_" + str(model_number) + "/train/metal/"
    train_perfect_folder = "potato_dataset/data_" + str(procent) + "/model_" + str(model_number) + "/train/perfect/"
    
    test_hollow_folder = "potato_dataset/data_" + str(procent) + "/model_" + str(model_number) + "/test/hollow/"
    test_metal_folder = "potato_dataset/data_" + str(procent) + "/model_" + str(model_number) + "/test/metal/"
    test_perfect_folder = "potato_dataset/data_" + str(procent) + "/model_" + str(model_number) + "/test/perfect/"

    train_data_hollow = []
    train_data_metal = []
    train_data_perfect = []

    test_data_hollow = []
    test_data_metal = []
    test_data_perfect = []

    
    
    # Training data
    for filename in os.listdir(train_hollow_folder):
        if filename != ".DS_Store" and filename != ".png" and filename != ".ipynb_checkpoints": 
            img = (cv2.imread(train_hollow_folder + filename, cv2.IMREAD_GRAYSCALE) / 255.0)
            train_data_hollow.append(img)


    for filename in os.listdir(train_metal_folder):
        if filename != ".DS_Store" and filename != ".png" and filename != ".ipynb_checkpoints":
            img = (cv2.imread(train_metal_folder + filename, cv2.IMREAD_GRAYSCALE) / 255.0)
            train_data_metal.append(img)


    for filename in os.listdir(train_perfect_folder):
        if filename != ".DS_Store" and filename != ".png" and filename != ".ipynb_checkpoints":
            img = (cv2.imread(train_perfect_folder + filename, cv2.IMREAD_GRAYSCALE) / 255.0)
            train_data_perfect.append(img)
    
    
    # Test data
    for filename in os.listdir(test_hollow_folder):
        if filename != ".DS_Store" and filename != ".png" and filename != ".ipynb_checkpoints":
            img = (cv2.imread(test_hollow_folder + filename, cv2.IMREAD_GRAYSCALE) / 255.0)
            test_data_hollow.append(img)


    for filename in os.listdir(test_metal_folder):
        if filename != ".DS_Store" and filename != ".png" and filename != ".ipynb_checkpoints":
            img = (cv2.imread(test_metal_folder + filename, cv2.IMREAD_GRAYSCALE) / 255.0)
            test_data_metal.append(img)


    for filename in os.listdir(test_perfect_folder):
        if filename != ".DS_Store" and filename != ".png" and filename != ".ipynb_checkpoints":
            img = (cv2.imread(test_perfect_folder + filename, cv2.IMREAD_GRAYSCALE) / 255.0)
            test_data_perfect.append(img)
            
            
    # Augmented training data
    for augmentation in augmented_data_list:
        if augmentation == "vae" or augmentation == "gan" or augmentation == "gan_2":
            augmented_hollow_folder = "potato_dataset/data_" + str(procent) + "/model_" + str(model_number) + "/" + augmentation + "/hollow/"    
            augmented_perfect_folder = "potato_dataset/data_" + str(procent) + "/model_" + str(model_number) + "/" + augmentation + "/perfect/"
            augmented_metal_folder = "potato_dataset/data_" + str(procent) + "/model_" + str(model_number) + "/" + augmentation + "/metal/"
        
            for filename in os.listdir(augmented_hollow_folder):
                if filename != ".DS_Store" and filename != ".png" and filename != ".ipynb_checkpoints":
                    img = (cv2.imread(augmented_hollow_folder + filename, cv2.IMREAD_GRAYSCALE) / 255.0)
                    train_data_hollow.append(img)
            
            for filename in os.listdir(augmented_metal_folder):
                if filename != ".DS_Store" and filename != ".png" and filename != ".ipynb_checkpoints":
                    img = (cv2.imread(augmented_metal_folder + filename, cv2.IMREAD_GRAYSCALE) / 255.0)
                    train_data_metal.append(img)

            for filename in os.listdir(augmented_perfect_folder):
                if filename != ".DS_Store" and filename != ".png" and filename != ".ipynb_checkpoints":
                    img = (cv2.imread(augmented_perfect_folder + filename, cv2.IMREAD_GRAYSCALE) / 255.0)
                    train_data_perfect.append(img)
        else:    
            augmented_hollow_folder = "potato_dataset/data_" + str(procent) + "/model_" + str(model_number) + "/" + augmentation + "/hollow/"
            augmented_metal_folder = "potato_dataset/data_" + str(procent) + "/model_" + str(model_number) + "/" + augmentation + "/metal/"
            augmented_perfect_folder = "potato_datset/data_" + str(procent) + "/model_" + str(model_number) + "/" + augmentation + "/perfect/"
        
            for filename in os.listdir(augmented_hollow_folder):
                if filename != ".DS_Store" and filename != ".png" and filename != ".ipynb_checkpoints":
                    img = (cv2.imread(augmented_hollow_folder + filename, cv2.IMREAD_GRAYSCALE) / 255.0)
                    train_data_hollow.append(img)

            for filename in os.listdir(augmented_metal_folder):
                if filename != ".DS_Store" and filename != ".png" and filename != ".ipynb_checkpoints":
                    img = (cv2.imread(augmented_metal_folder + filename, cv2.IMREAD_GRAYSCALE) / 255.0)
                    train_data_metal.append(img)


            for filename in os.listdir(augmented_perfect_folder):
                if filename != ".DS_Store" and filename != ".png" and filename != ".ipynb_checkpoints":
                    img = (cv2.imread(augmented_perfect_folder + filename, cv2.IMREAD_GRAYSCALE) / 255.0)
                    train_data_perfect.append(img)

    # Convert lists to np array
    np_train_data_hollow = np.array(train_data_hollow)
    np_train_data_metal = np.array(train_data_metal)
    np_train_data_perfect = np.array(train_data_perfect)

    # Convert lists to np array
    np_test_data_hollow = np.array(test_data_hollow)
    np_test_data_metal = np.array(test_data_metal)
    np_test_data_perfect = np.array(test_data_perfect)

    # Data labels
    np_train_data_hollow_labels = np.full(len(np_train_data_hollow), 0)
    np_train_data_metal_labels = np.full(len(np_train_data_metal), 1)
    np_train_data_perfect_labels = np.full(len(np_train_data_perfect), 2)

    np_test_data_hollow_labels = np.full(len(np_test_data_hollow), 0)
    np_test_data_metal_labels = np.full(len(np_test_data_metal), 1)
    np_test_data_perfect_labels = np.full(len(np_test_data_perfect), 2)


    # Concatenate train and label data
    train_data =  np.concatenate((np_train_data_hollow, np_train_data_metal, np_train_data_perfect), axis=0) 
    train_labels = np.concatenate((np_train_data_hollow_labels, np_train_data_metal_labels, np_train_data_perfect_labels), axis=0)
 
    test_data = np.concatenate((np_test_data_hollow, np_test_data_metal, np_test_data_perfect), axis=0) 
    test_labels = np.concatenate((np_test_data_hollow_labels, np_test_data_metal_labels, np_test_data_perfect_labels), axis=0)

    train_data = train_data.reshape(train_data.shape[0], 128, 128, 1)
    test_data = test_data.reshape(test_data.shape[0], 128, 128, 1)

    return train_data, train_labels, test_data, test_labels

# Define network

In [3]:
def run_model(train_data, train_labels, test_data, test_labels, patience):
    model = Sequential()
    #add model layers
    model.add(Conv2D(8, kernel_size=3, padding='same', strides=2, activation=tf.nn.relu, input_shape=(128,128,1)))
    model.add(Conv2D(8, kernel_size=3, padding='same', strides=2, activation=tf.nn.relu))
    model.add(Conv2D(8, kernel_size=3, padding='same', strides=2, activation=tf.nn.relu))
    #model.add(Conv2D(32, kernel_size=3, padding='same', strides=2, activation=tf.nn.relu))
    model.add(Flatten())
    model.add(Dense(1024, activation=tf.nn.relu))
    model.add(Dense(512, activation=tf.nn.relu))
    model.add(Dense(3, activation=tf.nn.softmax))

    model.compile(optimizer='adam', 
                  loss='sparse_categorical_crossentropy', 
                  metrics=['accuracy'])
    #model.summary()
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=patience)


    history = model.fit(x=train_data,y=train_labels, validation_data=(test_data, test_labels), shuffle=True, batch_size=32, epochs=1000, verbose=0, callbacks=[es])
    history_acc = np.array(history.history['val_acc'])
    acc_mean = np.mean(history.history['val_acc'][-patience:])
    _, train_acc = model.evaluate(train_data, train_labels, verbose=0)
    _, test_acc = model.evaluate(test_data, test_labels, verbose=0)
    ynew = model.predict_classes(test_data)
    K.clear_session()
    return acc_mean, ynew, train_acc, test_acc


In [ ]:
# Define augmentation
augmentaion_list = []

for p in range(10, 100, 10):
    start = time.time()
    final_acc = []
    model_acc = 0
    hollow_hollow = 0
    hollow_metal = 0
    hollow_perfect = 0
    metal_hollow = 0
    metal_metal = 0
    metal_perfect = 0
    perfect_hollow = 0
    perfect_metal = 0
    perfect_perfect = 0
    sum_hollow = 0
    sum_metal = 0
    sum_perfect = 0
    for j in range(10):
        model_acc = 0
        train_data, train_labels, test_data, test_labels = get_train_and_test_data(p, j + 1, augmentaion_list)
        #print(train_data.shape)
        for i in range(3):
            acc_mean, pred_labels, train_acc, test_acc = run_model(train_data, train_labels, test_data, test_labels, 40 + p)
            print("Run {} with model {} got train acc {} on test acc {}".format(i, j + 1, train_acc, test_acc))
            for m in range(len(test_labels)):

                if test_labels[m] == 0:
                    sum_hollow += 1

                    if pred_labels[m] == 0:
                        hollow_hollow += 1
                    elif pred_labels[m] == 1:
                        hollow_metal += 1
                    elif pred_labels[m] == 2:
                        hollow_perfect +=1

                elif test_labels[m] == 1:
                    sum_metal += 1
                    if pred_labels[m] == 0:
                        metal_hollow += 1
                    elif pred_labels[m] == 1:
                        metal_metal += 1
                    elif pred_labels[m] == 2:
                        metal_perfect +=1

                elif test_labels[m] == 2:
                    sum_perfect += 1
                    if pred_labels[m] == 0:
                        perfect_hollow += 1
                    elif pred_labels[m] == 1:
                        perfect_metal += 1
                    elif pred_labels[m] == 2:
                        perfect_perfect +=1


            model_acc += test_acc
        final_acc.append(model_acc)

    for i in range(1):
        print("Model {} got an acc on: {}".format(i, final_acc[i] / 3.0))
    
    print("Using {} procent of data to training our model got an acc {}".format(p, sum(final_acc) / 30))
    print("Hollow to hollow: {}".format(hollow_hollow / sum_hollow * 100))
    print("Hollow to metal: {}".format(hollow_metal / sum_hollow * 100))
    print("Hollow to perfect: {}".format(hollow_perfect / sum_hollow * 100))
    print("Metal to hollow: {}".format(metal_hollow / sum_metal * 100))
    print("Metal to metal: {}".format(metal_metal / sum_metal * 100))
    print("Metal to perfect: {}".format(metal_perfect / sum_metal * 100))
    print("Perfect to hollow: {}".format(perfect_hollow / sum_perfect * 100))
    print("Perfect to metal: {}".format(perfect_metal / sum_perfect * 100))
    print("Perfect to perfect: {}".format(perfect_perfect / sum_perfect * 100))
    end = time.time()
    print(end - start)